Тип данных Category

ПРИЗНАКИ: КАТЕГОРИАЛЬНЫЕ И ЧИСЛОВЫЕ

Под числовыми признаками обычно подразумевают признаки, которые отражают количественную меру и могут принимать значения из неограниченного диапазона.

Числовые признаки могут быть:

дискретными (например, количество комнат, пациентов, дней, отток сотрудников);
непрерывными (например, масса, цена, площадь).
Дискретные признаки чаще всего представлены целыми числами, а непрерывные — целыми числами и числами с плавающей точкой.

Под категориальными признаками обычно подразумевают столбцы в таблице, которые обозначают принадлежность объекта к какому-то классу/категории.

Категориальные признаки могут быть:

номинальными (например, пол, национальность, район);
порядковыми (например, уровень образования, уровень комфорта, стадия заболевания).
Такие признаки имеют ограниченный набор значений. Они чаще всего представлены в виде текстового описания и кодируются в Pandas типом данных object.

Однако это не всегда так. Например, созданный нами ранее признак месяца продажи кодируется числом (от 1 до 12), но на самом деле является категориальным, поскольку диапазон его значений ограничен и каждому числу мы можем поставить в соответствие название месяца.

КАТЕГОРИИ В ДАННЫХ О НЕДВИЖИМОСТИ

Давайте определим число уникальных категорий в каждом столбце нашей таблицы melb_df. Для этого создадим вспомогательную таблицу unique_counts:

In [1]:
import pandas as pd
melb_df = pd.read_csv('data/melb_data_ps.csv')

In [2]:
# создаём пустой список
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
    # создаём кортеж (имя столбца, число уникальных значений)
    item = (col, melb_df[col].nunique(), melb_df[col].dtypes) 
    # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
# выводим её на экран
display(unique_counts)

,Column_Name,Num_Unique,Type
0,Type,3,object
1,Method,5,object
2,Regionname,8,object
3,Rooms,9,int64
4,Bathroom,9,int64
5,Car,11,int64
6,Bedroom,12,int64
7,CouncilArea,33,object
8,Date,58,object
9,YearBuilt,144,int64


Итак, с помощью такого несложного кода мы смогли получить вспомогательную таблицу, где в отсортированном виде показано количество уникальных значений во всех столбцах и их типы.

?
Что интересного мы можем узнать из такой таблицы?

Если присмотреться внимательно, можно увидеть резкий скачок количества уникальных значений, начиная с 14 строки таблицы, где число уникальных значений составляет 152. Учтём этот момент.

Условимся, что категориальными будем считать признаки, у которых число уникальных категорий меньше 150. 

Однако учтём, что признак Date (дата продажи), преобразованный нами ранее в формат datetime, является временным признаком, поэтому далее не будем его воспринимать как категориальный. 

ТИП ДАННЫХ CATEGORY

→ Этот тип данных является гибридным: внешне он выглядит как строка, но внутренне представлен массивом целых чисел. Так как данные вместо изначальных строк хранятся в памяти как число, то объём памяти, занимаемой таблицей при использовании типа category, резко уменьшается, что повышает эффективность хранения и работы с таблицей.

→ Более того, этот тип данных расширяет возможности работы с категориальными признаками: мы можем легко преобразовывать категории, строить графики по таким данным (что сложно сделать для типа данных object). Также резко повышается производительность операций, совершаемых с такими столбцами.

In [ ]:
# Сделаем преобразование столбцов к типу данных category:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
display(melb_df.info())

ПОЛУЧЕНИЕ АТРИБУТОВ CATEGORY

У типа данных category есть свой специальный аксесcор cat, который позволяет получать информацию о своих значениях и преобразовывать их. Например, с помощью атрибута этого аксессора categories мы можем получить список уникальных категорий в столбце Regionname:

In [ ]:
melb_df['Regionname'].cat.categories

С помощью метода аксессора rename_categories() можно легко переименовать текущие значения категорий. Для этого в данный метод нужно передать словарь, ключи которого — старые имена категорий, а значения — новые.

Рассмотрим на примере: переименуем категории признака типа постройки Type — заменим их на полные названия (напомним, u — unit, h — house, t — townhouse).

In [ ]:
melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})
display(melb_df['Type'])

тип данных category хранит только категории, которые были объявлены при его инициализации. При встрече с новой, неизвестной ранее категорией, этот тип превратит её в пустое значение, так как он просто не знает о существовании этой категории.

Решить эту проблему на самом деле не сложно. Можно добавить категорию flat в столбец Type с помощью метода акссесора cat add_categories(), в который достаточно просто передать имя новой категории:

In [ ]:
melb_df['Type'] = melb_df['Type'].cat.add_categories('flat')
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
display(new_houses_types)

Теперь, когда мы рассмотрели основные моменты и нюансы работы с типом данных category, можно сформулировать несколько рекомендаций по его использованию:

1
Необязательно каждый раз преобразовывать категориальные данные в тип данных category. Зачастую это делается исключительно для оптимизации работы с большими данными.

2
Если набор данных занимает значительный процент используемой оперативной памяти, рассмотрите возможность использования типа category.

3
Если у вас очень серьёзные проблемы с производительностью, обратите внимание на использование типа category.

4
Если вы решили использовать тип category, будьте осторожны при добавлении новой информации в вашу таблицу. Убедитесь, что вы собрали всю необходимую информацию, произведите предобработку данных и только после этого используйте преобразование типов.